In [1]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [2]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [3]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')
loansContract = web3.eth.contract(address=loansContractAddress, abi=loansContractABI)

In [4]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0x1fa7a3a7814b018e0a5b96e938b0a037246c7a1f82d2997c8244d007ebffb741')

HexBytes('0x52523875563b89b16256f61969f65b8c5416923639b1c049e636017dda9f9c6c')

In [5]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0xf96dd37fd53375b8d041040b713c14d87585d5558cdb2100c8f48f4b5db433c4')

HexBytes('0x6391548563929b36d6f00d99075a87fc5b54ccdc7492a465d8855d9bd9fbf5ab')

HexBytes('0xa814703ae35f00a6c871e5efb3f91a4cdd05ed3a13249542346ea37287c83b1b')

HexBytes('0x1f6c1860234d4c10ec3c17b375372a41e8bc5da23985f2b8d8739e8f43b50928')

In [6]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_loanie, _loaner, _amount, _organizationContract):
    
    transaction = _organizationContract.functions.createLoan(_loanie, _amount 
    ).buildTransaction({
    'gas': 3000000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    }) 
    _privateKey = getpass("Enter the password of the organization: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    
    # _organizationContract.functions.createLoan(_loanie, _loaner, _amount).transact()
    return True

In [99]:
deleteUser()

In [7]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [9]:
# Suppose bank wants to create a loan
#loaner = input("Enter the loaner addrses: ")
loaner = web3.eth.accounts[5]
loaner
### Check to see whether it is a user or organization address
loanerIndex = accountsConract.functions.getIndex(loaner).call()

'0xDED00A7f19D445B2c6D3ddFffbf78BB490771937'

In [10]:
#loanie = input("Enter the loanie address: ")
loanie = web3.eth.accounts[2]
loanie
loanieIndex = accountsConract.functions.getIndex(loanie).call()

'0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290'

## Get pending loans

In [18]:
# Get pending loans using 3 separate functions
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [19]:
getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)

Enter your password: ········


['78117 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1589449772 ',
 '85739 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1589449776 ',
 '86564 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1589449778 ',
 '38264 0xDED00A7f19D445B2c6D3ddFffbf78BB490771937 1589449780 ']

## Confirm or reject loans

In [21]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)
private_key = getpass('Enter your loanie password: ')
for pendingLoan in pendingLoans:
    print('for id: ' + pendingLoan.split(' ')[0])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 300000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 300000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

Enter your password: ········
Enter your loanie password: ········
for id: 86564
c/r?r
for id: 38264
c/r?r


# Create a new loan

In [17]:
loaner
# Create a new loan
if loanerIndex != -1:
    loanieType = accountsConract.functions.getType(loanerIndex).call()
    if loanieType:
        if createLoan(_loanie=web3.eth.accounts[2], _loaner=loaner, _amount=random.randint(1,100000), _organizationContract=organizationContract):
            print("Loan created")
else:
    print("This account is not registered in our system.")

'0xDED00A7f19D445B2c6D3ddFffbf78BB490771937'

Enter the password of the organization: ········
Loan created


In [23]:
loans=loansContract.functions.getLoans().call()
print(loans)

[(1589449772, '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290', '0xDED00A7f19D445B2c6D3ddFffbf78BB490771937', 78117), (1589449776, '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290', '0xDED00A7f19D445B2c6D3ddFffbf78BB490771937', 85739)]
